In [1]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2022-08-07 10:43:33--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 74.125.134.136, 74.125.134.93, 74.125.134.91, ...
Connecting to dl.google.com (dl.google.com)|74.125.134.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12415 (12K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  12.12K  --.-KB/s    in 0s      

2022-08-07 10:43:33 (42.4 MB/s) - ‘linux_signing_key.pub’ saved [12415/12415]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]    
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]       
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]    
Get:6 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [109

In [2]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2022-08-07 10:44:01--  http://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 74.125.31.128, 2607:f8b0:400c:c02::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|74.125.31.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7067070 (6.7M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   6.74M  --.-KB/s    in 0.03s   

2022-08-07 10:44:01 (239 MB/s) - ‘/tmp/chromedriver.zip’ saved [7067070/7067070]

Archive:  /tmp/chromedriver.zip
  inflating: /usr/local/bin/chromedriver  


In [3]:
# install selenium
!sudo apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.4 MB of archives.
After this operation, 154 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.1 [494 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd

In [4]:
# To check Chrome Driver's version
!chromedriver -v

ChromeDriver 104.0.5112.79 (3cf3e8c8a07d104b9e1260c910efb8f383285dc5-refs/branch-heads/5112@{#1307})


In [5]:
# import libraries
import pandas as pd
import numpy as np
import re

from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
# creating the driver object and loading the url
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=chrome_options)

In [7]:
# scrapping function , returns dictionary

def scrap_twitter_data(url):
    influencer = np.NaN
    influncer_tweet = np.NaN
    influencer_tweet_likes = np.NaN
    influencer_tweet_text = np.NaN
    time_stamp = np.NaN
    promoter = np.NaN
    promoter_tweet = np.NaN
    promoter_tweet_likes = np.NaN
    promoter_tweet_text = np.NaN

    try:
        # loading the twitter url
        driver.get(url)
        driver.implicitly_wait(20)

        # Influencer Details
        try:
            # if this elemnt exists then influencer tweet has been deleted
            element = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/div/article//child::a/span')

        except:
            # Scrapping influencer url
            element = driver.find_elements_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//preceding::article[1]//child::a')
            influencer = element[0].get_attribute('href')

            # Scrapping influncer_tweet url
            influncer_tweet = element[3].get_attribute('href')

            # Scrapping influencer_tweet_likes
            element = driver.find_elements_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//preceding::article[1]//child::div[@role="group"]//child::span')
            influencer_tweet_likes = element[-1].text

            # influencer_tweet_text
            element = driver.find_elements_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//preceding::article[1]//child::div[@lang="en"]//child::span')
            influencer_tweet_text = ''
            for i in element:
                influencer_tweet_text += i.text

        # Promoter Details

        # Scrapping time_stamp
        element = driver.find_element_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article/div/div/div/div[3]/div[6]/div/div[1]/a[1]/span')
        time_stamp = element.text   

        # Scrapping promoter url
        element = driver.find_elements_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//child::a')
        promoter = element[0].get_attribute('href')

        # Scrapping promoter_tweet url
        promoter_tweet = url   

        # Scrapping promoter_tweet_likes
        try:
            element = driver.find_element_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//child::div[@role="group"]//child::span//child::span//child::span')
            promoter_tweet_likes = element.text
        except:
            pass

        # promoter_tweet_text
        element = driver.find_elements_by_xpath('//span[contains(text(),"Twitter for Android" ) or contains(text(),"Twitter Web App" )]//ancestor::article//child::div[@lang="en"]//child::span')
        promoter_tweet_text = ''
        for i in element:
            promoter_tweet_text += i.text
    except:
        pass
    
    # Creating dictionary
    res = {}
    res['time_stamp'] = time_stamp
    res['influencer'] = influencer
    res['promoter'] = promoter
    res['influncer_tweet'] = influncer_tweet
    res['promoter_tweet'] = promoter_tweet
    res['influencer_tweet_likes'] = influencer_tweet_likes
    res['promoter_tweet_likes'] = promoter_tweet_likes
    res['influencer_tweet_text'] = influencer_tweet_text
    res['promoter_tweet_text'] = promoter_tweet_text
    
    return res

In [8]:
# getting all url links

df = pd.read_csv('../input/twitter-links/links.csv',skiprows=2,names=['index','links']).drop('index',axis = 1)
df = df.dropna()
df = df.set_index('links').reset_index()
df.head()

,links
0,https://twitter.com/psalmcrypt/status/14424604...
1,https://twitter.com/psalmcrypt/status/14424654...
2,https://twitter.com/psalmcrypt/status/14425121...
3,https://twitter.com/psalmcrypt/status/14425192...
4,https://twitter.com/psalmcrypt/status/14425217...


In [9]:
# initializing variables
# scraping done in batches

arr = []
j = 664

In [10]:
while j<df.shape[0]:
    arr.append(scrap_twitter_data(df.links[j]))
    j+=1
    print(f'{j}--->',end='>')

665--->>666--->>667--->>668--->>669--->>670--->>671--->>672--->>673--->>674--->>675--->>676--->>677--->>678--->>679--->>680--->>681--->>682--->>683--->>684--->>685--->>686--->>687--->>688--->>689--->>690--->>691--->>692--->>693--->>694--->>695--->>696--->>697--->>698--->>699--->>700--->>701--->>702--->>703--->>704--->>705--->>706--->>707--->>708--->>709--->>710--->>711--->>712--->>713--->>714--->>715--->>716--->>717--->>718--->>719--->>720--->>721--->>722--->>723--->>724--->>725--->>726--->>727--->>728--->>729--->>730--->>731--->>732--->>733--->>734--->>735--->>736--->>737--->>738--->>739--->>740--->>741--->>742--->>743--->>744--->>745--->>746--->>747--->>748--->>749--->>750--->>751--->>752--->>753--->>754--->>755--->>756--->>757--->>758--->>759--->>760--->>761--->>762--->>763--->>764--->>765--->>766--->>767--->>768--->>769--->>770--->>771--->>772--->>773--->>774--->>775--->>776--->>777--->>778--->>779--->>780--->>781--->>782--->>783--->>784--->>785--->>786--->>787--->>788--->>789--->>

In [14]:
# creating dataframe
dff = pd.DataFrame(arr)

In [18]:
# exporting the dataframe in form of csv file
dff.to_csv('Twitter_data_02.csv',index=False)

# Creating final dataset
- Datas are scrapped in batches so its time to join them

In [20]:
t1 = pd.read_csv('../input/twitter-scrapped-data/Twitter_data.csv')
t2 = pd.read_csv('../input/twitter-scrapped-data/Twitter_data_02.csv')

In [23]:
# creating final dataset
final_dataset = t1.append(t2)
final_dataset.head()

,time_stamp,influencer,promoter,influncer_tweet,promoter_tweet,influencer_tweet_likes,promoter_tweet_likes,influencer_tweet_text,promoter_tweet_text
0,"12:05 PM · Sep 27, 2021",https://twitter.com/gokhshteinmedia,https://twitter.com/psalmcrypt,https://twitter.com/gokhshteinmedia/status/144...,https://twitter.com/psalmcrypt/status/14424604...,29,1.0,Which happens first?#crypto #bitcoin $ETH,power play period for #BTC next the next 6 mo...
1,"12:25 PM · Sep 27, 2021",https://twitter.com/davidgokhshtein,https://twitter.com/psalmcrypt,https://twitter.com/davidgokhshtein/status/144...,https://twitter.com/psalmcrypt/status/14424654...,867,1.0,Super Bullish.,"It's a bullish week, everyone woke up with the..."
2,"3:31 PM · Sep 27, 2021",https://twitter.com/davidgokhshtein,https://twitter.com/psalmcrypt,https://twitter.com/davidgokhshtein/status/144...,https://twitter.com/psalmcrypt/status/14425121...,679,1.0,"$XRP, $XDC and $XLM.\n\nThe X’s. \n\nIf you di...","Good for you David, I'm really not a fan of th..."
3,"3:59 PM · Sep 27, 2021",https://twitter.com/davidgokhshtein,https://twitter.com/psalmcrypt,https://twitter.com/davidgokhshtein/status/144...,https://twitter.com/psalmcrypt/status/14425192...,70,2.0,"So Jaime, you’re saying #Bitcoin is legit?",David were you thinking #BTC is a Ponzi scheme...
4,"4:09 PM · Sep 27, 2021",https://twitter.com/davidgokhshtein,https://twitter.com/psalmcrypt,https://twitter.com/davidgokhshtein/status/144...,https://twitter.com/psalmcrypt/status/14425217...,118,2.0,Are you stacking more $BTC or $ETH? \n\nSo far...,"I also think the same for $ETH, $ETH 2.0 will ..."


In [24]:
# exporting final dataset
final_dataset.to_csv('Final_twitter_dataset.csv',index = False)